## Q1 What's the version.build_hash value?

In [1]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")
es_client.info()

ObjectApiResponse({'name': 'e4b5a5e14825', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'zyXlNsJYQLy7ZS25PgkrRw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [2]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

## ans.1 '42f05b9372a9a4a470db3b52817899b99a76ee73'

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
len(documents)

948

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Q2. Which function do you use for adding your data to elastic?

In [5]:
##use tqdm if you want to see the time bar
#for doc in documents:
#    es_client.index(index=index_name, document=doc)

## ans.2 index

## Q3. what's the score for the top ranking result?

In [7]:
index_name = "course-questions"

user_question = "How do I execute a command in a running docker container?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

In [8]:
response = es_client.search(index=index_name, body=search_query)

for hit in response['hits']['hits']:
    doc = hit['_source']
    print(f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n")

Section: 5. Deploying Machine Learning Models
Question: How do I debug a docker container?
Answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)


Section: Module 1: Docker and Terraform
Question: PGCLI - running in a Docker container
Answer: In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.
Below the usage with values used in the videos of the course for:
network name (docker network)
postgres related variables for pgcli
Hostname
Username
Port
Database name
$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1
175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi
Password for root:
Server: PostgreSQL 1

In [9]:
for hit in response['hits']['hits']:
    score = hit['_score']
    print(score)

84.050095
75.54128
72.08518
51.04628
49.938507


## ans.3 score = 84.05

## Q4. What's the 3rd question returned by the search engine?

In [10]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}


response = es_client.search(index=index_name, body=search_query)
#response

In [11]:
for hit in response['hits']['hits']:
    doc = hit['_source']
    print(f"Question: {doc['question']}\n")

Question: How do I debug a docker container?

Question: How do I copy files from my local machine to docker container?

Question: How do I copy files from a different folder into docker container’s working directory?



## ans.4 How do I copy files from a different folder into docker container’s working directory?

## Q5. What's the length of the resulting prompt? (use the len function)

In [18]:
def retrieve_documents(query, index_name="course-questions", max_results=3):
    es_client = Elasticsearch("http://localhost:9200")
    
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    documents = [hit['_source'] for hit in response['hits']['hits']]
    return documents

In [12]:
user_question

'How do I execute a command in a running docker container?'

In [13]:
for hit in response['hits']['hits']:
    doc = hit['_source']
    print(doc)

{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}
{'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani", 'section': '5. Deploying Machine Learning Models', 'question': 'How do I copy files from my local machine to docker container?', 'course': 'mac

In [14]:
result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

result_docs

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [21]:
context_template = """
Q: {question}
A: {text}
""".strip()

context = ""

for doc in result_docs:
    context = context + context_template.format(question = doc['question'], text = doc['text']) + '\n\n'

context

'Q: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani\n\nQ: How do I copy files from a different folder into docker container’s working directory?\nA: You can copy files from your local machine into a Docker container using the docker cp comman

In [22]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [23]:
prompt = prompt_template.format(question=user_question, context=context).strip()
prompt

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do I execute a command in a running docker container?\n\nCONTEXT:\nQ: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_dir

In [24]:
len(prompt)

1462

## ans.5 1462

## Q6. How many tokens does our prompt have?

In [25]:
import tiktoken

In [26]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [27]:
prompt_tokens = encoding.encode(prompt)
len(prompt_tokens)

322

## ans.6 322

In [29]:
prompt_tokens[0:5]

[63842, 261, 4165, 14029, 29186]

In [31]:
encoding.decode_single_token_bytes(29186)

b' assistant'

## generating the answer

In [32]:
from openai import OpenAI

In [34]:
client = OpenAI()

In [35]:
openai_response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

openai_response.choices[0].message.content

'To execute a command in a running Docker container, you can use the `docker exec` command. Here are the steps to do so:\n\n1. First, find the container ID of the running container by using the command:\n   ```\n   docker ps\n   ```\n\n2. Next, execute the desired command in the container using the container ID. For example, to start a bash session, you would use:\n   ```\n   docker exec -it <container-id> bash\n   ```'

## calculating the costs

In [37]:
in_tokens = len(prompt_tokens)
in_tokens

322

In [38]:
openai_response

ChatCompletion(id='chatcmpl-9fSedaROtYT8wgcdZCrTs0uiY2MNc', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='To execute a command in a running Docker container, you can use the `docker exec` command. Here are the steps to do so:\n\n1. First, find the container ID of the running container by using the command:\n   ```\n   docker ps\n   ```\n\n2. Next, execute the desired command in the container using the container ID. For example, to start a bash session, you would use:\n   ```\n   docker exec -it <container-id> bash\n   ```', role='assistant', function_call=None, tool_calls=None))], created=1719668623, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_ce0793330f', usage=CompletionUsage(completion_tokens=98, prompt_tokens=329, total_tokens=427))

In [43]:
response_tokens = encoding.encode(openai_response.choices[0].message.content)
out_tokens = len(response_tokens)
out_tokens

98

In [44]:
#1000 rerequests
cost = 0.005*(1000*322)/1000 + 0.015*(1000*98)/1000
print("cost:", cost, " $")

cost: 3.08  $
